# Tensorflow のバッチ推論を SageMaker で行う

#### ノートブックに含まれる内容

- Tensorflow のバッチ推論を SageMaker で行うやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: MNIST
- データ: CNN

## セットアップ

In [ ]:
import os
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.session import Session

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## データのロード

Tensorflow 向けに，protobuf フォーマットの TFRecord に変換して，変換済データを S3 にアップロードします．SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，MNIST データをダウンロードしてからローカルにある `utils.py` 変換処理を行って，S3 にアップロードします．

デフォルトでは SageMaker は sagemaker-{region}-{your aws account number} というバケットを使用します．当該バケットがない場合には，自動で新しく作成します．upload_data() メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することが可能です．

In [ ]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

以下を実行する前に，**<span style="color: red;">`data/mnist/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')

## モデルの学習を実行

学習プロセスは，通常の SageMaker での Tensorflow の使い方と変わりはありません．Tensorflow オブジェクトを作って，`fit()` で学習するだけです．

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             training_steps=3, 
                             evaluation_steps=2,
                             train_instance_count=1,
                             train_instance_type='ml.m4.xlarge')

mnist_estimator.fit(inputs)

## モデルのバッチ推論を実行

学習が終わったら，`deploy()` による通常のエンドポイント作成と異なり，バッチ推論の場合は `Transformer` オブジェクトを作成します．作成したオブジェクトに対して，実際に `transfor()` を実行することではじめて，裏側でインスタンスが立ち上がりバッチ推論が実行されます．

なお，ここでは `estimator.transformer()` で `Transformer` オブジェクトを作成しましたが，直接モデル名を指定して，`Transformer()` でオブジェクトを初期化することも可能です．詳細については [SageMaker Python SDK の github リポジトリ](https://github.com/aws/sagemaker-python-sdk#sagemaker-batch-transform)を参照してください．

In [ ]:
transformer = mnist_estimator.transformer(instance_count=1, instance_type='ml.m4.xlarge')

バッチ推論を行う際には，S3 上のデータを入力として指定します．なおここでは省略していますが，出力については `Transformer` の初期化の際に指定することが可能です．詳細については [API Doc](https://sagemaker.readthedocs.io/en/latest/transformer.html#sagemaker.transformer.Transformer) をご覧ください．

In [ ]:
input_bucket_name = 'sagemaker-sample-data-{}'.format(boto3.Session().region_name)
input_file_name = 'data.csv'
input_file_path = 'batch-transform/mnist/' + input_file_name
test_data = 's3://{}/{}'.format(input_bucket_name, input_file_path)

transformer.transform(test_data, content_type='text/csv')

上記の `transform()` は非同期処理のメソッドですので，SageMaker に対してバッチ推論ジョブをリクエストしたら終了します．これを明示的にジョブの終了まで待つ場合には，`wait()` メソッドをご使用ください．

In [ ]:
transformer.wait()

### バッチ推論の結果を確認

上記のバッチが終わったら，S3 の出力先のパスを確認した上でダウンロードして，結果を確認しましょう．

In [ ]:
print(transformer.output_path)

あとは通常の `boto3` クライアントを使って，手元にファイルをダウンロードして，表示してみます．

In [ ]:
import boto3
import json
from urllib.parse import urlparse

s3 = boto3.resource('s3')

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], input_file_name)

output_obj = s3.Object(bucket_name, file_key)
output = output_obj.get()["Body"].read()

print('Prediction is {}'.format(json.loads(output)['outputs']['classes']['int64Val']))